In [2]:
import sqlite3
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
file_path = '../data/tv/tv_live_1.csv'

In [22]:
df_tv = pd.read_csv(file_path, delimiter=';', thousands=' ', decimal=',', dtype={
                    'Month': 'category', 'Week': 'category', 'Weekday': 'category',
                    'Dayparts': 'object', 'Channel Groups': 'object', 
                    'Channel': 'object', 'PIB pos': 'int8', 'PIB (real) rel': 'object',
                    'PIB count': 'int16', 'Dur rounded,sp': 'int8', 'Spot Class': 'object',
                    'Block Code': 'object', 'Syndicate': 'object', 'Producer': 'object',
                    'Brand': 'object', 'Film Code': 'object', 'Prog Campaign': 'object',
                    'Prog Before': 'object', 'Prog After': 'object', 
                    'Film Code/2': 'object'}, 
                    parse_dates=['Date', 'Time'], date_format='%d.%m.%Y'
                    )
df_tv['DateTime'] = df_tv['Date'].astype('str') + ' ' + df_tv['Time']
df_tv['DateTime'] = pd.to_datetime(df_tv['DateTime'], format='ISO8601')
df_tv[['Dayparts', 'Prog Before', 'Prog After']] = df_tv[['Dayparts', 'Prog Before', 'Prog After']].fillna('brak danych', axis=1)
df_tv.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 972606 entries, 0 to 972605
Data columns (total 24 columns):
 #   Column                                      Non-Null Count   Dtype         
---  ------                                      --------------   -----         
 0   Month                                       972606 non-null  category      
 1   Week                                        972606 non-null  category      
 2   Weekday                                     972606 non-null  category      
 3   Date                                        972606 non-null  datetime64[ns]
 4   Time                                        972606 non-null  object        
 5   Dayparts                                    972606 non-null  object        
 6   Channel Groups                              972606 non-null  object        
 7   Channel                                     972606 non-null  object        
 8   PIB pos                                     972606 non-null  int8         

In [24]:
df_tv.head(2)

,Month,Week,Weekday,Date,Time,Dayparts,Channel Groups,Channel,PIB pos,PIB (real) rel,PIB count,"Dur rounded,sp",Spot Class,Block Code,Syndicate,Producer,Brand,Film Code,Prog Campaign,Prog Before,Prog After,Film Code/2,TRP All 40+ cities 200- (i wsie) Nat[TSV2],DateTime
0,202001,202001,Wed,2020-01-01,02:00:05,Off stacji,TVP tem,TVP Seriale,1,First,10,30,Commercial,Between,Media expert sp z oo,Media expert sp z oo,MEDIA EXPERT,Media expert sklep 1599zl czyszczenie magazyno...,PILKARSKI POKER /FILM POL./,no data,Reklama dzieciom - final akcji,259128,0.05,2020-01-01 02:00:05
1,202001,202001,Wed,2020-01-01,02:02:33,Off stacji,Polsat tem,Fokus TV,1,First,10,30,Commercial,Between,Media expert sp z oo,Media expert sp z oo,MEDIA EXPERT,Media expert sklep 1599zl czyszczenie magazyno...,GWIAZDY LOMBARDU /REALITY SHOW/,Gwiazdy lombardu,Infokus,259134,0.41,2020-01-01 02:02:33
